## Imports

In [12]:
import numpy as np
import os
import torch
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

## Pre-Processcing

In [4]:
cat_dir = 'train\\cats'
dog_dir = 'train\\dogs'

cat_filenames = os.listdir(cat_dir)
dog_filenames = os.listdir(dog_dir)

train = []
labels = [0]*10000 + [1]*10000

In [5]:

for i,path in enumerate(cat_filenames):
    if i%10000 == 0:
        print(i//10000)
    im = cv2.imread(os.path.join(cat_dir,path))
    im = cv2.resize(im ,(128,128))
    train.append(im)

for i,path in enumerate(dog_filenames):
    if i%10000 == 0:
        print(i//10000)
    im = cv2.imread(os.path.join(dog_dir,path))
    im = cv2.resize(im ,(128,128))
    train.append(im)



0
0


In [15]:
#labels = [0]*10000 + [1]*10000
train = np.array(train, dtype=np.float32)  # Ensure correct data type
train = torch.tensor(train).permute(0, 3, 1, 2)  # Convert to PyTorch format (N, C, H, W)

labels = torch.tensor(labels)

dataset = TensorDataset(train, labels)
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

## Model 

In [13]:
model = nn.Sequential(
    nn.Conv2d(3,16,5),# 3*128*128 => 16*124*124
    nn.ReLU(),
    nn.BatchNorm2d(16),
    nn.MaxPool2d(2,2),# 16*124*124 => 16*62*62

    nn.Conv2d(16,32,3),# 16*62*62 => 32*60*60
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*60*60 => 32*30*30

    nn.Conv2d(32,32,3),# 32*30*30 => 32*28*28
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2,2),# 32*28*28 => 32*14*14

    nn.Conv2d(32,32,5),# 32*14*14 => 32*10*10
    nn.ReLU(),
    nn.BatchNorm2d(32),

    nn.Flatten(),

    nn.Linear(32*10*10,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    
    nn.Linear(128,32),
    nn.ReLU(),
    nn.BatchNorm1d(32),
    
    nn.Linear(32,2)
)

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

## Model-Train

In [28]:
epoch=3
for i in range(epoch):
    epoch_loss = 0
    correct = 0
    for batch_images , batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_images)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()


        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()
    
    accuracy = 100 * correct / len(train)
    print(f"Epoch {i+1}, Accuracy: {accuracy:.2f}%")

Epoch 1, Accuracy: 85.89%
Epoch 2, Accuracy: 88.60%
Epoch 3, Accuracy: 90.64%


## Testing Model

In [40]:
from functools import partial
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [36]:
test_cats_dir='test\\cats'
test_dogs_dir='test\\dogs'

cat_testfiles = os.listdir(test_cats_dir)
dog_testfiles = os.listdir(test_dogs_dir)

test = []
labels = [0]*2500 + [1]*2500

In [37]:
for i,path in enumerate(cat_testfiles):
    
    im = cv2.imread(os.path.join(test_cats_dir,path))
    im = cv2.resize(im ,(128,128))
    test.append(im)
    if i%2500 == 0:
        print("cats are into test")

for i,path in enumerate(dog_testfiles):
    im = cv2.imread(os.path.join(test_dogs_dir,path))
    im = cv2.resize(im ,(128,128))
    test.append(im)
    if i%2500 == 0:
        print("dogs are also into test")

cats are into test
dogs are also into test


In [38]:
len(test),len(labels)

(5000, 5000)

In [39]:
test = np.array(test, dtype=np.float32)  # Ensure correct data type
test = torch.tensor(test).permute(0, 3, 1, 2)  # Convert to PyTorch format (N, C, H, W)

labels = torch.tensor(labels)

test_dataset = TensorDataset(test, labels)
batch_size = 32
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## main-Test

In [42]:
model.eval()  
all_preds = []
all_labels = []
with torch.no_grad():  
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")

Test Accuracy: 0.8756
Test Precision: 0.9023
Test Recall: 0.8424
Test F1-Score: 0.8713
